In [29]:
%pip install numpy

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)



[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [1]:
import numpy as np
import transformers
from transformers import AutoModelForTokenClassification, TrainingArguments, Trainer, AutoTokenizer, CamembertForTokenClassification, DataCollatorForTokenClassification, CamembertTokenizerFast
from datasets import load_dataset, load_metric, load_dataset, ClassLabel, DownloadConfig

import os
from pathlib import Path

from tokenizer import Tokenizer
from trajet_dataset import IOBTRAJETDataset

/Users/gautierpamies/Documents/Epitech/T-AIA-901-MPL_7/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/Users/gautierpamies/Documents/Epitech/T-AIA-901-MPL_7/tokenizer.py:7: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("seqeval", trust_remote_code=True)


In [2]:
# !python -m spacy download fr_core_news_sm

In [2]:
metric = load_metric("seqeval", trust_remote_code=True)

In [3]:
def compute_metrics(p, label_list):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    # Remove ignored index (special tokens)
    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = metric.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

In [4]:
model_n_version = "trajet_v2"
max_epochs = 6
learning_rate = 2e-5
batch_size = 32

In [5]:
pretrained_model_checkpoint = "camembert-base"
pretrained_tokenizer_checkpoint = "camembert-base"

In [7]:
# decomment if you don't have datas
# !python generate_data.py

In [6]:
dataset = IOBTRAJETDataset()


/Users/gautierpamies/Documents/Epitech/T-AIA-901-MPL_7/.venv/lib/python3.11/site-packages/huggingface_hub/repocard.py:105: UserWarning: Repo card metadata block was not found. Setting CardData to empty.
  warnings.warn("Repo card metadata block was not found. Setting CardData to empty.")


In [7]:
preprocessor = Tokenizer.init_vf(pretrained_tokenizer_checkpoint=pretrained_tokenizer_checkpoint)

In [8]:
model = CamembertForTokenClassification.from_pretrained(pretrained_model_checkpoint,
                                                            num_labels=len(dataset.labels))
    

Some weights of CamembertForTokenClassification were not initialized from the model checkpoint at camembert-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [9]:
model.config.id2label = dataset.id2label

print(model.config.id2label)
model.config.label2id = dataset.label2id
print(model.config.label2id)

{0: 'B-DEP', 1: 'B-DEST', 2: 'I-DEP', 3: 'I-DEST', 4: 'O'}
{'B-DEP': 0, 'B-DEST': 1, 'I-DEP': 2, 'I-DEST': 3, 'O': 4}


In [10]:
tokenized_datasets = dataset.dataset.map(preprocessor.tokenize_and_align_labels, 
                                         batched=True,
                                         load_from_cache_file=True)

Map: 100%|██████████| 29796/29796 [00:06<00:00, 4576.18 examples/s]


In [11]:
args = TrainingArguments(
        f"test-ner",
        evaluation_strategy="epoch",
        learning_rate=learning_rate,
        per_device_train_batch_size=batch_size,
        per_device_eval_batch_size=batch_size,
        num_train_epochs=max_epochs,
        weight_decay=0.01, 
        gradient_accumulation_steps=4,
        gradient_checkpointing=True
    )

In [12]:
data_collator = DataCollatorForTokenClassification(tokenizer=preprocessor.tokenizer, padding=False)

In [13]:
len(tokenized_datasets["train"])

59593

In [14]:
len(tokenized_datasets["validation"])

29796

In [15]:
trainer = Trainer(
        model.to('cpu'),
        args,
        train_dataset=tokenized_datasets["train"],
        eval_dataset=tokenized_datasets["validation"],
        data_collator=data_collator,
        tokenizer=preprocessor.tokenizer,
        compute_metrics=lambda p: compute_metrics(p=p, label_list=dataset.labels),
    )

In [18]:
trainer.train()

  0%|          | 0/2790 [00:00<?, ?it/s]/Users/gautierpamies/Documents/Epitech/T-AIA-901-MPL_7/.venv/lib/python3.11/site-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
                                                         
 17%|█▋        | 465/2790 [4:44:08<16:06:59, 24.95s/it]

{'eval_loss': 0.0044873119331896305, 'eval_precision': 0.9995794714797558, 'eval_recall': 0.9996131007452016, 'eval_f1': 0.9995962858296326, 'eval_accuracy': 0.9999428452378099, 'eval_runtime': 1185.9046, 'eval_samples_per_second': 25.125, 'eval_steps_per_second': 0.786, 'epoch': 1.0}


 18%|█▊        | 500/2790 [4:57:10<14:27:33, 22.73s/it]  

{'loss': 0.0107, 'learning_rate': 1.6415770609318996e-05, 'epoch': 1.07}


/Users/gautierpamies/Documents/Epitech/T-AIA-901-MPL_7/.venv/lib/python3.11/site-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
                                                          
 33%|███▎      | 931/2790 [11:30:55<11:56:16, 23.12s/it]

{'eval_loss': 0.0022312377113848925, 'eval_precision': 0.99973086627418, 'eval_recall': 0.9997813178125052, 'eval_f1': 0.9997560914068478, 'eval_accuracy': 0.9999594385558651, 'eval_runtime': 2225.031, 'eval_samples_per_second': 13.391, 'eval_steps_per_second': 0.419, 'epoch': 2.0}


 36%|███▌      | 1000/2790 [11:57:37<11:41:57, 23.53s/it] 

{'loss': 0.0032, 'learning_rate': 1.2831541218637992e-05, 'epoch': 2.15}


/Users/gautierpamies/Documents/Epitech/T-AIA-901-MPL_7/.venv/lib/python3.11/site-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
 36%|███▌      | 1005/2790 [11:59:37<11:52:08, 23.94s/it]

RuntimeError: MPS backend out of memory (MPS allocated: 3.67 GB, other allocations: 14.16 GB, max allowed: 18.13 GB). Tried to allocate 384.00 MB on private pool. Use PYTORCH_MPS_HIGH_WATERMARK_RATIO=0.0 to disable upper limit for memory allocations (may cause system failure).

In [18]:
PYTORCH_MPS_HIGH_WATERMARK_RATIO=0.0 


In [16]:
from transformers import AutoModelForTokenClassification, TrainingArguments, Trainer, AutoTokenizer, CamembertForTokenClassification, DataCollatorForTokenClassification, CamembertTokenizerFast
model_path = "./models/trajet_v2"
tokenizer = CamembertTokenizerFast.from_pretrained(model_path)
model = CamembertForTokenClassification.from_pretrained(model_path)

In [19]:
trainer.evaluate()

100%|██████████| 932/932 [19:59<00:00,  1.29s/it]


{'eval_loss': 0.0020520715042948723,
 'eval_precision': 0.9997645040286632,
 'eval_recall': 0.9997981395192356,
 'eval_f1': 0.9997813214910509,
 'eval_accuracy': 0.9999594385558651,
 'eval_runtime': 1200.975,
 'eval_samples_per_second': 24.81,
 'eval_steps_per_second': 0.776}

In [20]:
predictions, labels, _ = trainer.predict(tokenized_datasets["test"])
predictions = np.argmax(predictions, axis=2)

100%|██████████| 932/932 [29:51<00:00,  1.92s/it]    


In [21]:
true_predictions = [
        [dataset.labels[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

true_labels = [
            [dataset.labels[l] for (p, l) in zip(prediction, label) if l != -100]
            for prediction, label in zip(predictions, labels)
        ]


In [22]:
true_predictions

[['O',
  'O',
  'O',
  'B-DEP',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'B-DEST',
  'I-DEST',
  'O'],
 ['O',
  'B-DEP',
  'O',
  'B-DEST',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O'],
 ['O',
  'O',
  'O',
  'B-DEP',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'B-DEST',
  'O',
  'O',
  'O',
  'O',
  'O'],
 ['O', 'O', 'O', 'B-DEP', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-DEST', 'O'],
 ['O', 'O', 'O', 'B-DEP', 'O', 'B-DEST', 'O', 'O', 'O', 'O', 'O', 'O', 'O'],
 ['O', 'B-DEP', 'I-DEP', 'I-DEP', 'O', 'O', 'B-DEST', 'O', 'O', 'O', 'O', 'O'],
 ['O',
  'O',
  'B-DEP',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'B-DEST',
  'O'],
 ['B-DEP',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'B-DEST',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O'],
 ['O', 'O', 'O', 'B-DEP', 'O', 'O', 'O'

In [23]:
results = metric.compute(predictions=true_predictions, references=true_labels)
print("*" *200)
print(results)
print("*" *200)


********************************************************************************************************************************************************************************************************
{'DEP': {'precision': 0.9996643732169828, 'recall': 0.9996308229292522, 'f1': 0.9996475977916128, 'number': 29796}, 'DEST': {'precision': 0.9996626180836707, 'recall': 0.9996963460305678, 'f1': 0.9996794817726344, 'number': 29639}, 'overall_precision': 0.9996634979389248, 'overall_recall': 0.9996634979389248, 'overall_f1': 0.9996634979389248, 'overall_accuracy': 0.9999373098552595}
********************************************************************************************************************************************************************************************************


In [26]:
import os
import json
import numpy as np

# Ensure the 'results' directory exists
results_directory = "results/" + model_n_version 
os.makedirs(results_directory, exist_ok=True)

# Specify the file path
results_file_path = os.path.join(results_directory, "results.json")

# Define a custom encoder function to handle int32 objects
def custom_encoder(obj):
    if isinstance(obj, np.int64):
        return int(obj)
    raise TypeError("Object not serializable")

# Save results to a JSON file with the custom encoder
with open(results_file_path, 'w') as results_file:
    json.dump(results, results_file, default=custom_encoder)

print(f"Results saved to: {results_file_path}")


Results saved to: results/trajet_v2/results.json


In [ ]:

# out_dir = os.path.expanduser("models") + "/" + model_n_version #models\trajet_v1
# trainer.save_model(out_dir)